In [19]:
from langgraph.graph import StateGraph, START, END
from typing_extensions import TypedDict, Annotated
from langgraph.types import Send
import operator

In [20]:
from typing import Union


class State(TypedDict):
    words: list[str]
    output: Annotated[list[dict[str, Union[str, int]]], operator.add]

graph_builder = StateGraph(State)

In [21]:
def node_first(state):
    print(f"I want to count {len(state["words"])} words in my state.")
    return {}

def node_second(word: str):
    return {
        "output": [
            {
                "word": word,
                "letters": len(word),
            }
        ]
    }

In [ ]:
graph_builder.add_node("node_first", node_first)
graph_builder.add_node("node_second", node_second)

def dispatcher(state: State):
    return [Send("node_second", word) for word in state["words"]]

graph_builder.add_edge(START, "node_first")
graph_builder.add_conditional_edges("node_first", dispatcher, ["node_second"],)
graph_builder.add_edge("node_second", END)

In [23]:
graph = graph_builder.compile()

graph.invoke({
    "words": ["hello", "world", "how", "are", "you", "doing"]
})

# [
#     {"word": "hello", "letters": 5},
#     {"word": "world", "letters": 5},
#     {"word": "how", "letters": 3},
#     ...
# ]

ValueError: Found edge starting at unknown node 'node_fourth'